<a href="https://colab.research.google.com/github/mikckel/Task2_Movie_Recommendation/blob/main/Movie_Recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
!pip install implicit


In [20]:
pip install scikit-surprise


In [21]:
import pandas as pd

# Load the dataset
ratings = pd.read_csv('ratings.csv')

# Check column names
print(ratings.columns)


Index(['userId', 'movieId', 'rating', 'timestamp'], dtype='object')


In [22]:
# Load movies dataset
movies = pd.read_csv('movies.csv')

# Check column names
print(movies.columns)

# Merge datasets to ensure 'title' is in the ratings DataFrame
data = pd.merge(ratings, movies, on='movieId')
print(data.head())


Index(['movieId', 'title', 'genres'], dtype='object')
   userId  movieId  rating   timestamp                title  \
0       1      296     5.0  1147880044  Pulp Fiction (1994)   
1       3      296     5.0  1439474476  Pulp Fiction (1994)   
2       4      296     4.0  1573938898  Pulp Fiction (1994)   
3       5      296     4.0   830786155  Pulp Fiction (1994)   
4       7      296     4.0   835444730  Pulp Fiction (1994)   

                        genres  
0  Comedy|Crime|Drama|Thriller  
1  Comedy|Crime|Drama|Thriller  
2  Comedy|Crime|Drama|Thriller  
3  Comedy|Crime|Drama|Thriller  
4  Comedy|Crime|Drama|Thriller  


In [23]:
import pandas as pd
from surprise import Dataset, Reader
from surprise import KNNBasic, SVD
from surprise.model_selection import train_test_split
from surprise import accuracy

# Load the dataset
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')

# Prepare data for surprise
reader = Reader(rating_scale=(0.5, 5.0))
dataset = Dataset.load_from_df(data[['userId', 'title', 'rating']], reader)


In [28]:
# Split data into training and test sets
trainset, testset = train_test_split(dataset, test_size=0.25)

# Use SVD for matrix factorization
model = SVD()
model.fit(trainset)

# Predict ratings for the test set
predictions = model.test(testset)
print(f'RMSE: {accuracy.rmse(predictions)}')

RMSE: 0.7957
RMSE: 0.795684161680029


In [25]:
def get_svd_recommendations(user_id, model, data, n_recommendations=5):
    # Create a list of all movie titles
    all_titles = data['title'].unique()

    # Predict ratings for all movies
    user_ratings = [model.predict(user_id, title) for title in all_titles]

    # Sort by predicted rating and get top recommendations
    recommendations = sorted(user_ratings, key=lambda x: x.est, reverse=True)

    # Return top n recommendations
    return [rec.iid for rec in recommendations[:n_recommendations]]


In [26]:
user_id = 1
svd_recommendations = get_svd_recommendations(user_id, model, data, n_recommendations=5)
print("Recommendations for User 1 using SVD:")
print(svd_recommendations)

Recommendations for User 1 using SVD:
['Planet Earth II (2016)', 'Christmas Story, A (1983)', 'Parasite (2019)', 'All Watched Over by Machines of Loving Grace (2011)', 'Hedgehog in the Fog (1975)']
